# Semantic Memory System with Cognitive Scoring + Fact Generation & Refinement (PLUS VERSION)

**Fases:**
- **Validação**: Update de scores (EMA), decay em memórias fracas, **geração/refinamento de fatos**, salvamento imediato
- **Teste**: Memória congelada (read-only)

# 0) Setup

### A) Imports

In [13]:
# Standard library imports
import ast
import os
import re
import time
import sys

# Add parent directory to path for imports
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', '..')))
# Add vectorstore & prompts folders to path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', 'vectorstore')))
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', 'prompts')))

# Third-party imports
import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# LangChain imports
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI

# Memory managers & metadata builders
from cognitive_memory_manager import CognitiveMemoryManager, build_metadata_semantic
from simple_vector_memory import SimpleVectorMemory, build_metadata_reflection

# Prompt templates
from templates import (
    SEMANTIC_TEMPLATE,
    SCORE_TEMPLATE,
    EXTRACT_FACTS_TEMPLATE,
    REFINE_FACT_TEMPLATE,
)

# # Local imports
from utils_notebook import (
    SemanticCleaner,
    calcular_metricas_memoria,
    format_choices,
    parse_simple_score,
    make_question
)
cleaner = SemanticCleaner()

load_dotenv()

True

### B) Language Models and Datasets

In [14]:
LABELS = ['A','B','C','D']
number = {'A':0,'B':1,'C':2,'D':3}

In [15]:
# Dataset do arc
valid_df = pd.read_csv("../../dataset/arc_challenge_valid_processed.csv")
test_df = pd.read_csv("../../dataset/arc_challenge_test_processed.csv")

In [16]:
# Modelos para extração e tratamento
gpt5_nano = ChatOpenAI(model='gpt-5-nano-2025-08-07',temperature=0)

In [17]:
phi2 = ChatOllama(model="phi", temperature=0)
phi2_creative = ChatOllama(model="phi", temperature=0.6)  # Para geração de fatos

### C) Prompts and Chains

In [31]:
semantic_prompt = PromptTemplate.from_template(SEMANTIC_TEMPLATE)
rag_chain = semantic_prompt | phi2

score_prompt = PromptTemplate.from_template(SCORE_TEMPLATE)
score_chain = score_prompt | phi2 | StrOutputParser()

extract_facts_prompt = PromptTemplate.from_template(EXTRACT_FACTS_TEMPLATE)
extract_facts_chain = extract_facts_prompt | gpt5_nano | StrOutputParser()

refine_fact_prompt = PromptTemplate.from_template(REFINE_FACT_TEMPLATE)
refine_fact_chain = refine_fact_prompt | phi2_creative | StrOutputParser()

def call_organizer(fact, temperature=0.1):
    """Limpa e resume um fato científico."""
    prompt = f"""### Instruction:
Extract ONLY one objective scientific sentence from the text below. 
NO introductions, NO explanations, NO conversational filler.

Text: "{fact.strip()}"
Scientific Fact (Write it in only one sentence):"""
    
    model = ChatOllama(model="phi", temperature=temperature)
    fact_cleaned = model.invoke(prompt).content.strip().strip('"').strip("'")
    fact_cleaned = cleaner.clean(fact_cleaned)
    return fact_cleaned

# 1) Utils Functions

### A) Context Formatter

In [32]:
def format_semantic_context(semantic_items, show_scores=False):
    if not semantic_items:
        return "No specific reference information found."

    formatted_parts = []
    formatted_parts.append("")
    
    for i, fact in enumerate(semantic_items, 1):
        raw_q = fact['metadata'].get('question', '')
        q_text = raw_q.split('\n')[0].strip()
        f_text = fact.get('content', '').strip()
        
        block = f"""
    * **Principle #{i}**
        * Context: "{q_text}"
        * Fact: "{f_text}"
"""
        formatted_parts.append(block)        
            
    return "\n".join(formatted_parts)

### B) Answer Extractor 

In [33]:
# TODO Essa função deve posteriormente ser dividida em funções menores, cada uma responsável por um aspecto específico do processo de formatação.
# TODO Migrar a função para um arquivo utils

import re
from typing import List, Optional

def extract_answer(
    small_llm_model, 
    model_text_output: str, 
    valid_labels: List[str] = None,
    debug: bool = False,
    question: Optional[str] = None,
) -> str:
    """
    Extração robusta baseada em padrões de alta confiança (Tier 1) 
    com fallback para LLM Juiz em caso de falha.
    """
    if valid_labels is None:
        valid_labels = ['A', 'B', 'C', 'D', 'E']
    
    if not model_text_output:
        return "N/A"
    
    text = model_text_output.strip()

    if "```python" in text or "def solution" in text:
        if debug: print("⚠ [CODE DETECTED] Enviando para LLM Judge.")
        return _llm_judge(small_llm_model, text, valid_labels, debug, question)

    strong_patterns = [
        r"\\boxed\s*\{\s*([A-H])\s*\}",
        r"(?:Final|Correct)\s+Answer\s*[:\-]?\s*(?:is)?\s*(?:Option)?\s*[\(\[]([A-H])[\)\]](?![a-z])",
        r"The\s+(?:correct\s+)?(?:answer|option|choice)\s+is\s*(?:Option)?\s*[:\-]?\s*[\(\[]([A-H])[\)\]](?![a-z])",
        r"(?:Final|Correct)\s+Answer\s*[:\-]\s*(?:is\s+)?(?:Option\s+)?([A-H])(?=\s|\.|,|!|\?|$)(?![A-Za-z])",
        r"(?:Therefore|Thus|Hence|So),\s*(?:the\s+answer\s+is\s*)?(?:Option)?\s*[\(\[]([A-H])[\)\]](?![a-z])",
        r"(?:^|\n)\s*Answer\s*:\s*([A-H])(?=\s|$|\.|\,)(?![A-Za-z])",
        r"\*\*([A-H])\*\*(?![a-z])",
        r"^([A-H])$"
    ]
    
    for pattern in strong_patterns:
        matches = list(re.finditer(pattern, text, re.IGNORECASE))
        if matches:
            # Pega a última ocorrência desse padrão específico
            last_match = matches[-1]
            candidate = last_match.group(1).upper()
            
            if candidate in valid_labels:
                if debug: 
                    print(f"✓ [REGEX] Padrão encontrado: '{pattern}' -> {candidate}")
                return candidate

    #  FALLBACK: LLM 
    # Se nenhum padrão forte foi encontrado
    # Deixe o LLM ler e decidir.
    
    if debug: print("✗ [REGEX] Falha nos padrões fortes. Chamando LLM Judge.")
    return _llm_judge(small_llm_model, text, valid_labels, debug, question)


def _llm_judge(
    small_llm_model, 
    text: str, 
    valid_labels: List[str],
    debug: bool = False,
    question: Optional[str] = None,
) -> str:
    """
    Juiz focado em INFERÊNCIA. Ele deve ler o raciocínio e mapear para a letra correta,
    mesmo que o modelo não tenha dito a letra explicitamente.
    """
    
    # Monta contexto apenas se a pergunta existir
    context_block = ""
    if question:
        context_block = f"""
### CONTEXT (Use this to infer the answer letter):
{question}
"""

    prompt = f"""You are an Answer Extraction Bot. 
Your ONLY job is to identify which option the following "Model Output" concluded is correct.

{context_block}

### Model Output to Analyze:
{text}

### Instructions:
1. Look for an explicit answer (e.g., "Answer: A").
2. If NO explicit letter is found, read the conclusion of the "Model Output" and match it against the Options in the Context. **Infer the letter.**
   - Example: If Context has "(A) 5 (B) 10" and Model Output says "The result is 10", you must output B.
3. Do NOT calculate or solve the problem yourself. Trust the "Model Output".
4. If the model refuses to answer or is unclear, return "E".

Output format: Just the single letter (A, B, C, D, or E). No other text."""

    try:
        response = small_llm_model.invoke(prompt) 
        output = response.content if hasattr(response, 'content') else str(response)
        clean_cand = re.sub(r"[^A-E]", "", output.strip().upper())
        
        if len(clean_cand) > 1:
            clean_cand = clean_cand[-1]
            
        if clean_cand in valid_labels:
            if debug: print(f"✓ [LLM JUIZ] Inferido: {clean_cand}")
            return clean_cand
            
        return "N/A"

    except Exception as e:
        if debug: print(f"✗ [ERRO JUIZ] {e}")
        return "N/A"

### C) Check Fact Validation

In [34]:
def is_invalid(question, fact, original_fact, check_size=True, check_content=True, 
               check_similarity=True, min_similarity=0.2, return_reason=False):
    """
    Valida se um fato científico gerado é inválido.
    Inclui verificação de drift semântico via Similaridade de Cosseno.
    """
    if not fact or not isinstance(fact, str):
        return (True, "Fato vazio ou inválido") if return_reason else True

    fact_lower = fact.lower()
    word_count = len(fact.split())
    
    # 1. Validações de Estrutura
    if check_size:
        size_checks = [
            (word_count > 100, f"Muito longo ({word_count} tokens)"),
            (word_count < 10, f"Muito curto ({word_count} tokens)"),
            ('\n' in fact, "Contém quebras de linha"),
            ('#' in fact or '_' in fact, "Contém caracteres especiais (#/_)"),
            (fact_lower == original_fact.lower(), "Idêntico ao original")
        ]
        for condition, reason in size_checks:
            if condition: return (True, reason) if return_reason else True

    # 2. Validações de Conteúdo
    if check_content:
        forbidden_terms = ['score', 'correct', 'answer', 'instruction:', 'critique:']
        for term in forbidden_terms:
            if term in fact_lower:
                return (True, f"Contém termo proibido: {term}") if return_reason else True

    # 3. Validação de Similaridade
    if check_similarity:
        try:
            vectorizer = TfidfVectorizer(stop_words=None)
            tfidf = vectorizer.fit_transform([fact_lower, question.lower()])
            sim = cosine_similarity(tfidf[0:1], tfidf[1:2])[0][0]
            
            if sim < min_similarity:
                reason = f"Baixa similaridade ({sim:.2f}). Possível alucinação."
                return (True, reason) if return_reason else True
        except Exception as e:
            return (True, "Erro no cálculo de similaridade") if return_reason else True

    return (False, None) if return_reason else False

### D) Score Facts

In [35]:
def extract_and_score_new_facts(question, response_text, is_correct, pred, target, row, manager):
    """
    Extrai fatos científicos da resposta do modelo, atribui scores e salva imediatamente.
    
    Returns:
        List[dict]: Lista de fatos com scores
    """
    try:
        # 1. Extração de fatos com GPT-5 nano
        facts_text = extract_facts_chain.invoke({
            "model_response": response_text,
            "question": question
        })
        
        # Parse fatos (separados por linha)
        raw_facts = [f.strip() for f in facts_text.split('\n') if f.strip()]
        if not raw_facts:
            return []
        
        # Limitar a 2 fatos
        raw_facts = raw_facts[:2]
        
        # 2. Limpar fatos
        cleaned_facts = []
        for fact in raw_facts:
            cleaned = cleaner.clean(fact)
            if cleaned and len(cleaned.split()) >= 10:  # Mínimo 10 palavras
                cleaned_facts.append(cleaned)
        
        if not cleaned_facts:
            return []
        
        # 3. Calcular scores para cada fato
        scored_facts = []
        choices = ast.literal_eval(row['choices'])
        
        # Proteção contra índices inválidos
        try:
            correct_txt = choices['text'][number[target]] if target in number else str(target)
        except (IndexError, KeyError):
            correct_txt = str(target)
        
        try:
            chosen_txt = choices['text'][number[pred]] if pred in number else str(pred)
        except (IndexError, KeyError):
            chosen_txt = str(pred)
        
        status = "SUCCESS" if is_correct else "FAILURE"
        
        for fact in cleaned_facts:
            try:
                score_response = score_chain.invoke({
                    "question": question,
                    "chosen": chosen_txt,
                    "correct": correct_txt,
                    "outcome_status": status,
                    "fact": fact,
                    "reasoning": response_text
                })
                
                reflection, score = parse_simple_score(score_response)
                
                scored_facts.append({
                    'content': fact,
                    'score': score,
                    'reflection': reflection,
                    'original_id': str(row['id']),
                    'question': question,
                    'correct_answer': target
                })
            except Exception as e:
                print(f"⚠️ Erro ao calcular score: {e}")
                continue
        
        # 4. Salvar fatos no vectorstore
        if scored_facts:
            new_memory_entries = []
            for fact_data in scored_facts:
                new_memory_entries.append({
                    'content': fact_data['content'],
                    'metadata': {
                        'original_id': fact_data['original_id'],
                        'question': fact_data['question'],
                        'correct_answer': fact_data['correct_answer'],
                        'score': fact_data['score'],
                        'frequency': 0,
                        'scientific_fact': fact_data['content'],
                        'origin': 'validation_generated',
                        'type': 'semantic_memory',
                        'initial_score': fact_data['score']
                    }
                })
            
            manager.add_new_memories(new_memory_entries, id_prefix='generated')
        
        return scored_facts
        
    except Exception as e:
        print(f"⚠️ Erro na extração de fatos: {e}")
        return []

### E) Fact Refinement

In [36]:
def refine_negative_facts(used_facts, is_correct, question, pred, target, row, manager, 
                          refine_threshold=0.3, verbose=False):
    """
    ✨ NEW: Refina fatos recuperados que contribuíram para erro.
    Aplica apenas quando houve falha (is_correct = False).
    
    Args:
        refine_threshold: Score abaixo do qual um fato será refinado (default: 0.3)
        verbose: Se True, mostra informações de debug
    
    Returns:
        int: Número de fatos refinados
    """
    if is_correct or not used_facts:
        return 0
    
    refined_count = 0
    candidates_count = 0
    choices = ast.literal_eval(row['choices'])
    
    # Proteção contra índices inválidos
    try:
        correct_txt = choices['text'][number[target]] if target in number else str(target)
    except (IndexError, KeyError):
        correct_txt = str(target)
    
    try:
        wrong_txt = choices['text'][number[pred]] if pred in number else str(pred)
    except (IndexError, KeyError):
        wrong_txt = str(pred)
    
    if verbose:
        print(f"\n🔍 Analisando {len(used_facts)} fatos para refinamento (threshold: {refine_threshold})...")
    
    for memory in used_facts:
        fact_text = memory['content']
        memory_score = memory.get('memory_strength', 0.0)
        
        # Refina apenas fatos com score baixo (< refine_threshold)
        if memory_score >= refine_threshold:
            if verbose:
                print(f"   ⊗ Fato ignorado (score {memory_score:.2f} >= {refine_threshold})")
            continue
        
        candidates_count += 1
        if verbose:
            print(f"   ✓ Candidato #{candidates_count} (score: {memory_score:.2f})")
        
        try:
            # Gerar reflexão/crítica sobre o fato
            score_response = score_chain.invoke({
                "question": question,
                "chosen": wrong_txt,
                "correct": correct_txt,
                "outcome_status": "FAILURE",
                "fact": fact_text,
                "reasoning": f"Used fact led to wrong answer: {wrong_txt}"
            })
            
            reflection, _ = parse_simple_score(score_response)
            
            # Tentar gerar fato refinado
            max_attempts = 3
            for attempt in range(max_attempts):
                new_fact_text = refine_fact_chain.invoke({
                    "question": question,
                    "correct_answer": correct_txt,
                    "old_fact": fact_text,
                    "reflection": reflection,
                    "wrong_answer": wrong_txt
                })
                
                clean_new_fact = call_organizer(new_fact_text, temperature=0.1)
                
                # Validar fato refinado
                q_c = f"{question} {correct_txt}"
                if not is_invalid(question=q_c, fact=clean_new_fact, original_fact=fact_text, 
                                check_size=True, check_content=True, check_similarity=True):
                    # Salvar fato refinado
                    manager.add_new_memories([{
                        'content': clean_new_fact,
                        'metadata': {
                            'original_id': str(row['id']),
                            'question': question,
                            'correct_answer': target,
                            'score': 0,  # Score neutro inicial
                            'frequency': 0,
                            'scientific_fact': clean_new_fact,
                            'origin': 'validation_refined',
                            'type': 'semantic_memory',
                            'initial_score': 0,
                            'refined_from': fact_text
                        }
                    }], id_prefix='refined')
                    
                    refined_count += 1
                    if verbose:
                        print(f"   ✅ Fato refinado com sucesso (tentativa {attempt+1})")
                    break
                elif verbose and attempt == max_attempts - 1:
                    print(f"   ❌ Falha após {max_attempts} tentativas")
                    
        except Exception as e:
            if verbose:
                print(f"   ⚠️ Erro ao refinar fato: {e}")
            continue
    
    if verbose:
        print(f"📊 Resultado: {candidates_count} candidatos, {refined_count} refinados\n")
    
    return refined_count

### D) Evaluation

In [37]:
def avaliar_dataset_cognitive_plus(
    df, 
    chain=rag_chain,
    manager_semantic=None,
    score_chain=score_chain,
    k_semantic=3,
    threshold_semantic=0.24,
    semantic_weight=0.7,
    update_memory=False,
    decay_batch_size=False,
    decay_frequency=50,
    extract_new_facts=False,  # ✨ NEW
    refine_facts=False,       # ✨ NEW
    backup_frequency=160,      # 💾 NEW
    backup_path=None,         # 💾 NEW
    desc="Cognitive Memory System PLUS"
):
    """
    Avalia dataset com sistema híbrido (semântica + reflexão) + geração/refinamento.
    
    Args:
        extract_new_facts: Se True, extrai fatos das respostas do modelo
        refine_facts: Se True, refina fatos com baixo score após erro
        backup_frequency: Salva backup a cada N questões
        backup_path: Caminho base para salvar backups
    """
    resultados = []
    acertos = 0
    total = len(df)
    decay_counter = 0
    backup_counter = 0
    
    # Contadores de geração/refinamento
    total_facts_generated = 0
    total_facts_refined = 0
    total_facts_errors = 0
    
    # Acumuladores globais
    global_sims_sem = []
    global_counts_sem = []
    
    erros = 0

    # 💾 Preparar diretório de backup
    if backup_path:
        backup_dir = os.path.dirname(backup_path)
        if backup_dir and not os.path.exists(backup_dir):
            os.makedirs(backup_dir, exist_ok=True)
    
    loop = tqdm(df.iterrows(), total=total, desc=desc)

    for idx, row in loop:
        try:
            full_question = make_question(row, inline=False)[0]
            question_for_retriever = re.sub(r'\([A-Z]\)\s*', '', row['question'])
            
            # Recuperar memórias semânticas (COM SCORE COGNITIVO)
            items_semantic, _ = manager_semantic.retrieve_ranked_memories(
                query=question_for_retriever, 
                k=k_semantic,
                threshold=threshold_semantic,
                semantic_weight=semantic_weight,
                show_scores=False
            )
            

            
            # Combinar contextos
            cognitive_context_formatted = format_semantic_context(items_semantic)

            count_sem, avg_sim_sem, raw_sims_sem = calcular_metricas_memoria(items_semantic)
            
            global_counts_sem.append(count_sem)
            global_sims_sem.extend(raw_sims_sem)

            # Invocar o modelo com retry mechanism
            #temperatures = [0]  # Temperaturas para tentar
            #pred = 'E'
            #response_text = ""
            
            #for attempt, temp in enumerate(temperatures):
            #    try:
                    # # Criar chain com temperatura específica
                    # current_model = ChatOllama(model="phi", temperature=temp)
                    # current_chain = cognitive_prompt | current_model
                    
            response_obj = chain.invoke({
                "question": full_question,
                "similar_facts": cognitive_context_formatted, 
            })

            response_text = response_obj.content if hasattr(response_obj, "content") else str(response_obj)
            
            pred = extract_answer(
                small_llm_model=gpt5_nano, 
                model_text_output=response_text,
                question=full_question,
            )

            if pred not in ['A', 'B', 'C', 'D']:
                pred = 'E'  # E de Error/Exception
                erros += 1
                    
                    # Se a extração foi bem-sucedida (pred != 'E'), sair do loop
                    # if pred != 'E':
                    #     if attempt > 0:
                    #         tqdm.write(f"✓ Q{idx+1}: Sucesso na tentativa {attempt+1} (temp={temp})")
                    #     break
                    # else:
                    #     if attempt < len(temperatures) - 1:
                    #         tqdm.write(f"⚠ Q{idx+1}: Extração falhou (temp={temp}), tentando novamente...")
                
             #   except Exception as e:
              #      tqdm.write(f"⚠ Q{idx+1}: Erro na tentativa {attempt+1}: {e}")
               #     if attempt == len(temperatures) - 1:
                #        raise
            
            target = row['answerKey']

            is_correct = (pred == target)
            acertos += int(is_correct)
            
            # === ✨ NEW: EXTRAÇÃO DE NOVOS FATOS ===
            if extract_new_facts and pred != 'E':
                try:
                    new_facts = extract_and_score_new_facts(
                        question=full_question,
                        response_text=response_text,
                        is_correct=is_correct,
                        pred=pred,
                        target=target,
                        row=row,
                        manager=manager_semantic
                    )
                    if new_facts:
                        total_facts_generated += len(new_facts)
                    else:
                        total_facts_errors += 1
                except Exception as e:
                    total_facts_errors += 1
                    tqdm.write(f"❌ Q{idx+1}: Erro na extração de fatos: {e}")
            
            # === UPDATE DE MEMÓRIA EXISTENTE ===
            if update_memory and items_semantic and pred != 'E':
                used_ids = [m['doc_id'] for m in items_semantic]
                
                # Calcular feedback scores
                feedback_scores = []
                choices = ast.literal_eval(row['choices'])
                
                try:
                    correct_txt = choices['text'][number[target]] if target in number else str(target)
                except (IndexError, KeyError):
                    correct_txt = str(target)
                
                try:
                    chosen_txt = choices['text'][number[pred]] if pred in number else str(pred)
                except (IndexError, KeyError):
                    chosen_txt = str(pred)
                
                status = "SUCCESS" if is_correct else "FAILURE"
                
                for memory in items_semantic:
                    fact_text = memory['content']
                    
                    try:
                        score_response = score_chain.invoke({
                            "question": full_question,
                            "chosen": chosen_txt,
                            "correct": correct_txt,
                            "outcome_status": status,
                            "fact": fact_text,
                            "reasoning": response_text
                        })
                        
                        _, score = parse_simple_score(score_response)
                        feedback_scores.append(score)
                    except:
                        feedback_scores.append(0)
                
                # Aplicar update EMA
                manager_semantic.update_memories_feedback(used_ids, feedback_scores)
            
            # === ✨ NEW: REFINAMENTO DE FATOS ===
            if refine_facts and not is_correct:
                refined = refine_negative_facts(
                    used_facts=items_semantic,
                    is_correct=is_correct,
                    question=full_question,
                    pred=pred,
                    target=target,
                    row=row,
                    manager=manager_semantic
                )
                total_facts_refined += refined
            
            # === DECAY PERIÓDICO ===
            if update_memory and decay_frequency > 0 and (idx + 1) % decay_frequency == 0:
                decay_counter += 1
                tqdm.write(f"🌙 [Q{idx+1}/{total}] Aplicando ciclo de decay #{decay_counter}...")
                manager_semantic.apply_decay_cycle(decay_threshold=0.0)
            
            # === 💾 BACKUP PERIÓDICO ===
            if backup_path and backup_frequency > 0 and (idx + 1) % backup_frequency == 0:
                backup_counter += 1
                df_partial = pd.DataFrame(resultados)
                timestamp = time.strftime("%Y%m%d_%H%M%S")
                backup_file = backup_path.replace('.csv', f'_backup.csv')
                df_partial.to_csv(backup_file, index=False)
                #tqdm.write(f"💾 [Q{idx+1}/{total}] Backup #{backup_counter} salvo: {backup_file}")
            
            # Atualiza barra de progresso
            acc_atual = (acertos / (loop.n + 1)) * 100
            loop.set_postfix(
                acc=f"{acc_atual:.2f}%", 
                sem=count_sem, 
                decay=decay_counter,
                gen=total_facts_generated,
                refine=total_facts_refined,
                err=total_facts_errors,
                bkp=backup_counter,
                erros=erros
            )

            resultados.append({
                'index': idx,
                'question': full_question,
                'retrieved_context': cognitive_context_formatted,
                'retrieved_count_semantic': count_sem,
                'avg_similarity_semantic': avg_sim_sem,
                'raw_output': response_text,
                'pred': pred,
                'target': target,
                'is_correct': is_correct,
                'source': desc,
                'erros': erros
            })

        except Exception as e:
            tqdm.write(f"Erro no índice {idx}: {e}")
            resultados.append({
                'index': idx, 
                'error': str(e), 
                'is_correct': False, 
                'retrieved_count_semantic': 0,
                'retrieved_count_reflection': 0, 
                'avg_similarity_semantic': 0.0,
                'avg_similarity_reflection': 0.0,
            })
    
    # === DECAY FINAL ===
    if update_memory and decay_batch_size:
        decay_counter += 1
        print(f"\n🌙 [FINAL] Aplicando ciclo de decay final (#{decay_counter})...")
        manager_semantic.apply_decay_cycle(decay_threshold=0.0)
    
    # Sumário final
    if update_memory:
        print(f"\n📊 SUMÁRIO DE MEMÓRIA:")
        print(f"   Ciclos de decay: {decay_counter}")
        print(f"   Fatos gerados: {total_facts_generated}")
        print(f"   Fatos refinados: {total_facts_refined}")
        print(f"   Erros na extração: {total_facts_errors}")
        print(f"   Memórias semânticas finais: {manager_semantic._collection.count()}")
    
    if backup_path and backup_counter > 0:
        print(f"💾 Total de backups salvos: {backup_counter}")
    
    return pd.DataFrame(resultados)

# 2) Construção do Bancos de Dados Semântico

In [38]:
scientific_facts = pd.read_csv("../../scientific_facts_expanded.csv")
scientific_facts = scientific_facts.fillna("N/A")

# Preparar dados semânticos
df_semantic = scientific_facts[scientific_facts['scientific_fact'] != "N/A"].copy()
df_semantic.drop_duplicates(subset=['scientific_fact'], keep='first', inplace=True)

In [40]:
FORCE_RESET = False  # True quando quisermos recriar os bancos do zero
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2") 

manager_semantic = CognitiveMemoryManager(
    db_path="vectorstores/semantic_refinement/chroma_semantic",
    embedding_model=embedding_model, alpha=0.1, decay_lambda=0.99, forget_threshold=-0.6
).init_from_dataframe(df=df_semantic, content_col='scientific_fact',
                      id_prefix='semantic', metadata_func=build_metadata_semantic, reset_db=FORCE_RESET)

📂 Carregando memória existente de: vectorstores/semantic_refinement/chroma_semantic
✅ Memória carregada com 3993 itens.


# 3) Testing in one question

In [41]:
i = 0
k_semantic = 3
row = test_df.iloc[i]

In [42]:
q_text = make_question(row, inline=False)[0]
question_clean = re.sub(r'\([A-Z]\)\s*', '', q_text)

# Recuperar memórias semânticas
items_semantic, _ = manager_semantic.retrieve_ranked_memories(
    query=question_clean, 
    k=k_semantic, 
    threshold=0.24,
    semantic_weight=0.9,
    show_scores=True
)

semantic_context = format_semantic_context(items_semantic, show_scores=True)

print("Question: \n", q_text)
print("\nSemantic Context: \n", semantic_context)

count_sem, avg_sim_sem, raw_sims_sem = calcular_metricas_memoria(items_semantic)
print("\nMétricas Semânticas: Count:", count_sem, "AVG_SIM:", avg_sim_sem)

Question: 
 An astronomer observes that a planet rotates faster after a meteorite impact. Which is the most likely effect of this increase in rotation?
(A) Planetary density will decrease.
(B) Planetary years will become longer.
(C) Planetary days will become shorter.
(D) Planetary gravity will become stronger.

Semantic Context: 
 

    * **Principle #1**
        * Context: "Why is it winter in North America when it is summer in South America?"
        * Fact: "Axial precession causes the Earth's axis to change its orientation relative to its orbit around the sun over a period of approximately 26,000 years."


    * **Principle #2**
        * Context: "Michael learned that the movement of Earth in the solar system causes changes that can be seen on the planet. Which change could be seen on Earth in the time it takes Earth to rotate once on its axis?"
        * Fact: "This apparent movement and day-night change are due to Earth's axial tilt and its revolution around the Sun."


    * *

In [43]:
import time 

a = time.time()
semantic_context = format_semantic_context(items_semantic, show_scores=False)
response_obj = rag_chain.invoke({
    "question": q_text,
    "similar_facts": semantic_context, 
})
print("Time to answer: ", round(time.time()-a,2))
response_text = response_obj.content if hasattr(response_obj, "content") else str(response_obj)

pred = extract_answer(
    small_llm_model=gpt5_nano, 
    model_text_output=response_text,
    question=q_text,
    debug=True
)

target = row['answerKey']
is_correct = (pred == target)

print("# Raw Response: \n", response_text)
print(f"# Alternative chosen: {pred} (Correta: {target})")
print(f"# Resultado: {'CORRETO' if is_correct else 'ERRO'}")

Time to answer:  3.63
✓ [REGEX] Padrão encontrado: '(?:Final|Correct)\s+Answer\s*[:\-]?\s*(?:is)?\s*(?:Option)?\s*[\(\[]([A-H])[\)\]](?![a-z])' -> C
# Raw Response: 
  
The correct answer is (C) Planetary days will become shorter.

Reasoning:
1. The rotation of a planet determines the length of its day, so an increase in rotation would result in shorter planetary days.
2. This change could be due to the impact causing a shift in the planet's mass distribution, which affects its rotational speed.
3. The other options are not supported by the given information.

# Alternative chosen: C (Correta: C)
# Resultado: CORRETO


In [44]:
# Teste de extração de fatos
new_facts = extract_and_score_new_facts(
    question=q_text,
    response_text=response_text,
    is_correct=is_correct,
    pred=pred,
    target=target,
    row=row,
    manager=manager_semantic
)

print(f"\n✨ Fatos extraídos: {len(new_facts)}")
for i, fact in enumerate(new_facts, 1):
    print(f"\nFato {i} (Score: {fact['score']}): {fact['content']}")


✨ Fatos extraídos: 2

Fato 1 (Score: 1): The length of a planet's day is determined by its rotation rate.

Fato 2 (Score: 1): Day length is inversely related to rotation rate; faster rotation yields shorter days.


# 4) Running

In [ ]:
# VALIDAÇÃO: Memória ATIVA + Geração/Refinamento
df_resultado_valid_semantic_plus = avaliar_dataset_cognitive_plus(
    df=valid_df, 
    chain=rag_chain,
    manager_semantic=manager_semantic,
    score_chain=score_chain,
    k_semantic=3,
    semantic_weight=0.7,
    update_memory=True,
    decay_batch_size=True,
    decay_frequency=100,
    extract_new_facts=True,   # ✨ ATIVA extração
    refine_facts=True,        # ✨ ATIVA refinamento
    backup_frequency=130,      # 💾 Backup a cada 25 questões
    backup_path="../../results/semantic_refinement_cognitive_valid.csv",  # 💾 Caminho do backup
    desc="Validação (Memory Active + Gen/Refine)"
)

In [ ]:
# TESTE: Memória CONGELADA (apenas recuperação)
df_resultado_test_semantic_plus = avaliar_dataset_cognitive_plus(
    df=test_df, 
    chain=rag_chain,
    manager_semantic=manager_semantic,
    score_chain=score_chain,
    k_semantic=3,
    semantic_weight=0.7,
    update_memory=False,
    decay_batch_size=False,
    extract_new_facts=False,  # ✨ DESATIVA extração
    refine_facts=False,       # ✨ DESATIVA refinamento
    backup_frequency=150,      # 💾 Backup a cada 150 questões
    backup_path="../../results/semantic_refinement_cognitive_test07.csv",  # 💾 Caminho do backup
    desc="Teste (Memory Frozen)"
)

Teste (Memory Frozen): 100%|██████████| 1172/1172 [40:41<00:00,  2.08s/it, acc=70.65%, bkp=7, decay=0, err=0, erros=5, gen=0, refine=0, sem=3] 

💾 Total de backups salvos: 7


In [89]:
df_resultado_test_semantic_plus.to_csv("../../results/semantic_refinement_cognitive_test.csv", index=False)
df_resultado_valid_semantic_plus.to_csv("../../results/semantic_refinement_cognitive_valid.csv", index=False)